In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
#importing the table that wil soon be exported to SQL for some analysis
reviews = pd.read_excel("data\\SQL ba_reviews.xlsx", "ba_reviews_clean")
reviews.drop(columns=["Departed (City)", "Arrived"], axis = 1, inplace=True)
reviews.head(5)

,header,author,date,Departed,traveller_type,seat_type,date_flown,recommended,trip_verified,rating,seat_comfort,cabin_staff_service,food_beverages,ground_service,value_for_money,entertainment
0,service was mediocre at best,Gary Storer,2023-10-03,United Kingdom,Couple Leisure,Economy Class,2023-01-10,no,Not Verified,2,2,3,1,2,2,-1
1,BA standards continue to decline,A Jensen,2023-10-02,United Kingdom,Business,Business Class,2023-01-09,no,Verified,2,2,1,2,1,1,-1
2,"won the race to the bottom""",John Rockett,2023-10-02,United Kingdom,Couple Leisure,Business Class,2023-01-09,no,Not Verified,2,2,3,2,1,1,-1
3,Not a reliable airline,Tatiana Bobrovskaya,2023-10-02,United Kingdom,Business,Economy Class,2023-01-10,no,Verified,3,4,4,2,1,1,-1
4,Very disappointed,Tom Slowbe,2023-09-28,United States,Couple Leisure,First Class,2023-01-09,no,Verified,1,1,4,4,3,2,3


In [8]:
#printing the dimensions of the table
print(reviews.shape)
#checking the data type and number of non-null values in each column
reviews.info(verbose=True)

(1324, 16)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   header               1324 non-null   object        
 1   author               1324 non-null   object        
 2   date                 1324 non-null   datetime64[ns]
 3   Departed             1324 non-null   object        
 4   traveller_type       1323 non-null   object        
 5   seat_type            1324 non-null   object        
 6   date_flown           1324 non-null   datetime64[ns]
 7   recommended          1324 non-null   object        
 8   trip_verified        1324 non-null   object        
 9   rating               1324 non-null   int64         
 10  seat_comfort         1324 non-null   int64         
 11  cabin_staff_service  1324 non-null   int64         
 12  food_beverages       1324 non-null   int64         
 13  ground_service       1

In [16]:
#checking if (date, author) is a candidate key for the table
reviews.groupby(by=["date", "author"]).nunique()

1324